<h2>Importando bibliotecas</h2>

In [1]:
import findspark
findspark.init()
import pyspark


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
import pandas as pd
import pyspark.pandas as ps

<h2> Criando sessão do spark + configurações de conexão com bucket </h2>

In [4]:
spark = SparkSession.builder.appName('Steam API - Tratamento').getOrCreate()

22/03/14 21:40:57 WARN Utils: Your hostname, moon resolves to a loopback address: 127.0.1.1; using 192.168.0.185 instead (on interface wlo1)
22/03/14 21:40:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/14 21:40:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
def load_config(spark_context: SparkContext):
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.EnvironmentVariableCredentialsProvider')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.path.style.access', 'true')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 'http://localhost:9000')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.connection.ssl.enabled', 'false')
    
load_config(spark.sparkContext)


<h2> Lendo dados do bucket </h2>

In [28]:
df = spark.read.json('s3a://kafka-bucket/topics/steam/*', multiLine=True)

In [29]:
df.printSchema()

root
 |-- appid: string (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- language: string (nullable = true)
 |-- last_played: long (nullable = true)
 |-- num_games_owned: long (nullable = true)
 |-- num_reviews: long (nullable = true)
 |-- playtime_forever: long (nullable = true)
 |-- playtime_last_two_weeks: long (nullable = true)
 |-- received_for_free: boolean (nullable = true)
 |-- recommendationid: string (nullable = true)
 |-- review: string (nullable = true)
 |-- steam_purchase: boolean (nullable = true)
 |-- steamid: string (nullable = true)
 |-- timestamp_created: long (nullable = true)
 |-- timestamp_updated: long (nullable = true)
 |-- voted_up: boolean (nullable = true)
 |-- votes_funny: long (nullable = true)
 |-- votes_up: long (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- written_during_early_access: boolean (nullable = true)



In [30]:
df.count()

3565

In [16]:
#df = df.to_pandas_on_spark()

In [31]:
df.show(5)

+------+-------------+--------+-----------+---------------+-----------+----------------+-----------------------+-----------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+--------+-----------+--------+-------------------+---------------------------+
| appid|comment_count|language|last_played|num_games_owned|num_reviews|playtime_forever|playtime_last_two_weeks|received_for_free|recommendationid|              review|steam_purchase|          steamid|timestamp_created|timestamp_updated|voted_up|votes_funny|votes_up|weighted_vote_score|written_during_early_access|
+------+-------------+--------+-----------+---------------+-----------+----------------+-----------------------+-----------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+--------+-----------+--------+-------------------+---------------------------+
|271590|            0| english| 1647297014|         

In [52]:
from pyspark.sql import functions as f
from pyspark.sql import types as t
df.withColumn('last_played', f.date_format(df.last_played.cast(dataType=t.TimestampType()), "yyyy-MM-dd"))

DataFrame[appid: string, comment_count: bigint, language: string, last_played: string, num_games_owned: bigint, num_reviews: bigint, playtime_forever: bigint, playtime_last_two_weeks: bigint, received_for_free: boolean, recommendationid: string, review: string, steam_purchase: boolean, steamid: string, timestamp_created: bigint, timestamp_updated: bigint, voted_up: boolean, votes_funny: bigint, votes_up: bigint, weighted_vote_score: string, written_during_early_access: boolean]

In [53]:
df.show(1)

+------+-------------+--------+-----------+---------------+-----------+----------------+-----------------------+-----------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+--------+-----------+--------+-------------------+---------------------------+
| appid|comment_count|language|last_played|num_games_owned|num_reviews|playtime_forever|playtime_last_two_weeks|received_for_free|recommendationid|              review|steam_purchase|          steamid|timestamp_created|timestamp_updated|voted_up|votes_funny|votes_up|weighted_vote_score|written_during_early_access|
+------+-------------+--------+-----------+---------------+-----------+----------------+-----------------------+-----------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+--------+-----------+--------+-------------------+---------------------------+
|271590|            0| english| 1647297014|         

In [54]:
df.withColumn('last_played', f.to_date(df.last_played.cast(dataType=t.TimestampType())))

DataFrame[appid: string, comment_count: bigint, language: string, last_played: date, num_games_owned: bigint, num_reviews: bigint, playtime_forever: bigint, playtime_last_two_weeks: bigint, received_for_free: boolean, recommendationid: string, review: string, steam_purchase: boolean, steamid: string, timestamp_created: bigint, timestamp_updated: bigint, voted_up: boolean, votes_funny: bigint, votes_up: bigint, weighted_vote_score: string, written_during_early_access: boolean]

In [55]:
df.show(1)

+------+-------------+--------+-----------+---------------+-----------+----------------+-----------------------+-----------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+--------+-----------+--------+-------------------+---------------------------+
| appid|comment_count|language|last_played|num_games_owned|num_reviews|playtime_forever|playtime_last_two_weeks|received_for_free|recommendationid|              review|steam_purchase|          steamid|timestamp_created|timestamp_updated|voted_up|votes_funny|votes_up|weighted_vote_score|written_during_early_access|
+------+-------------+--------+-----------+---------------+-----------+----------------+-----------------------+-----------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+--------+-----------+--------+-------------------+---------------------------+
|271590|            0| english| 1647297014|         

In [58]:
from pyspark.sql import functions as f
from pyspark.sql.functions import col,lit
from datetime import datetime

df002 = df.withColumn("last_played",f.to_timestamp(df['last_played']))
    

In [60]:
df002.show(2)

+------+-------------+--------+-------------------+---------------+-----------+----------------+-----------------------+-----------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+--------+-----------+--------+-------------------+---------------------------+
| appid|comment_count|language|        last_played|num_games_owned|num_reviews|playtime_forever|playtime_last_two_weeks|received_for_free|recommendationid|              review|steam_purchase|          steamid|timestamp_created|timestamp_updated|voted_up|votes_funny|votes_up|weighted_vote_score|written_during_early_access|
+------+-------------+--------+-------------------+---------------+-----------+----------------+-----------------------+-----------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+--------+-----------+--------+-------------------+---------------------------+
|271590|            0| engli